In [ ]:
import pandas as pd
import numpy as np

In [ ]:
!pip -qqq install visualkeras
import visualkeras


In [ ]:
df = pd.read_csv('/kaggle/input/fake-news/FakeNewsNet.csv')
#df = df[['title', 'real']]
df = df[:6000]
df

In [ ]:
df['tweet'] = df['title'] + df['text']
df

In [ ]:
df = pd.read_csv('/kaggle/input/financial-sentiment-analysis/data.csv')
#df.columns = ['label', '0', '1', '2', '3', 'text']
df


In [ ]:
df.drop(columns = ['0', '1', '2', '3'], inplace = True)
df.head()

In [ ]:
df.isna().sum()

In [ ]:
import re
import string
import re
import nltk
#import contractions
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = stopwords.words('english')
from nltk.stem import PorterStemmer
ps = PorterStemmer()
exclude = string.punctuation
def preprocess(text):
    text = text.lower()
    #text = ' '.join(TextBlob(text).correct())
    text = text.translate(str.maketrans('','', exclude))
    text = re.sub('\d', '', text)
    text = re.sub(r'rt', '', text)
    #text = ' '.join([word for word in text.split() if word not in stop_words])
    text = re.sub(r'http\S+', '', text)
    text = re.sub(r'www\S+', '', text)
    #text = contractions.fix(text)
    #text = ' '.join(ps.stem(word) for word in text.split())
    return text
    

In [ ]:
df['clean'] = df['title'].apply(preprocess)

In [ ]:
df.head()

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Embedding, LSTM, Flatten, Concatenate, Attention
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
#clustered attention
# Tokenize the text data
max_words = 5000
tokenizer = Tokenizer(num_words=max_words, oov_token='<OOV>')
tokenizer.fit_on_texts(df['clean'])
X = tokenizer.texts_to_sequences(df['clean'])
X = pad_sequences(X)

# Encode categorical labels
label_encoder = LabelEncoder()
df['encoded_label'] = label_encoder.fit_transform(df['real'])
num_classes = len(label_encoder.classes_)

# Apply KMeans clustering on the tokenized text embeddings
embedding_dim = 32  # Adjust as needed
num_clusters = 2  # Number of clusters
kmeans = KMeans(n_clusters=num_clusters)
embeddings = tf.keras.layers.Embedding(input_dim=max_words, output_dim=embedding_dim)(X)
flat_embeddings = tf.keras.layers.Flatten()(embeddings)
clustered_embeddings = kmeans.fit_predict(flat_embeddings)


In [ ]:
clustered_embeddings.shape

In [ ]:
# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, df['encoded_label'], test_size=0.2, random_state=42)


In [ ]:
X_train.shape

In [ ]:
# 1. Create an Autoencoder Model
def create_autoencoder_model(input_dim, encoding_dim):
    inputs = Input(shape=(input_dim,))
    encoded = Dense(encoding_dim, activation='relu')(inputs)
    decoded = Dense(input_dim, activation='sigmoid')(encoded)
    autoencoder = Model(inputs=inputs, outputs=decoded)
    autoencoder.compile(optimizer='adam', loss='mse')
    return autoencoder

# Reshape clustered_embeddings to a 2D array
clustered_embeddings = clustered_embeddings.reshape(-1, 1)


autoencoder = create_autoencoder_model(clustered_embeddings.shape[1], encoding_dim=32)
autoencoder.fit(clustered_embeddings, clustered_embeddings, epochs=10, batch_size=16)


# 3. Obtain the Encoded Representations
encoded_embeddings = autoencoder.predict(clustered_embeddings)
encoded_embeddings = encoded_embeddings[:y_train.shape[0]]

In [ ]:
encoded_embeddings.shape

In [ ]:

# Define a simple transformer model with normal attention

import tensorflow as tf
from tensorflow.keras.layers import Dense, Input, Embedding, Flatten
from tensorflow.keras.optimizers import Adam

# Define a custom Attention layer with clustering on k and q matrices
class ClusteringAttention(tf.keras.layers.Layer):
    def __init__(self, num_clusters, **kwargs):
        super(ClusteringAttention, self).__init__(**kwargs)
        self.num_clusters = num_clusters
        self.q_dense = Dense(embedding_dim)
        self.k_dense = Dense(embedding_dim)
        self.built = False

    def build(self, input_shape):
        if not self.built:
            super(ClusteringAttention, self).build(input_shape)
            self.built = True

    def call(self, inputs):
        q, k, v = inputs

        # Apply KMeans clustering on q and k matrices
        k_clustered = self.k_dense(k)
        q_clustered = self.q_dense(q)

        # Perform attention mechanism
        attention_scores = tf.matmul(q_clustered, k_clustered, transpose_b=True)
        attention_scores = tf.nn.softmax(attention_scores)

        # Apply attention scores to value matrix
        output = tf.matmul(attention_scores, v)
        return output

    def compute_output_shape(self, input_shape):
        return input_shape[-1]

# Define a simple transformer model with clustered attention
def create_model(attention_layer):
    inputs = Input(shape=(X.shape[1],))
    embedding_layer = Embedding(input_dim=max_words, output_dim=embedding_dim)(inputs)
    q = Dense(embedding_dim)(embedding_layer)
    k = Dense(embedding_dim)(embedding_layer)
    v = Dense(embedding_dim)(embedding_layer)

    if attention_layer == 'clustering':
        # Create clustering layers
        k_clustered_layer = Dense(num_clusters, use_bias=False)
        q_clustered_layer = Dense(num_clusters, use_bias=False)

        # Apply clustering layers to q and k
        k_clustered = k_clustered_layer(k)
        q_clustered = q_clustered_layer(q)

        attention_layer = ClusteringAttention(num_clusters)([q_clustered, k_clustered, v])
    else:
        attention_layer = tf.keras.layers.Attention()([q, k, v])

    output_layer = Dense(num_classes, activation='softmax')(Flatten()(attention_layer))

    model = Model(inputs=inputs, outputs=output_layer)
    model.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model


def create_normal_attention_model():
    inputs = Input(shape=(X.shape[1],))
    embedding_layer = Embedding(input_dim=max_words, output_dim=embedding_dim)(inputs)
    q = Dense(embedding_dim)(embedding_layer)
    k = Dense(embedding_dim)(embedding_layer)
    v = Dense(embedding_dim)(embedding_layer)

    attention_layer = tf.keras.layers.Attention()([q, k, v])
    output_layer = Dense(num_classes, activation='softmax')(Flatten()(attention_layer))

    model = Model(inputs=inputs, outputs=output_layer)
    model.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Define a simple transformer model with clustered+autoencoder attention
def create_clustered_autoencoder_attention_model():
    inputs = Input(shape=(encoded_embeddings.shape[1],))
    q = Dense(encoded_embeddings.shape[1])(inputs)
    k = Dense(encoded_embeddings.shape[1])(inputs)
    v = Dense(encoded_embeddings.shape[1])(inputs)

    # Clustering Attention Layer
    attention_layer = ClusteringAttention(num_clusters)([q, k, v])

    output_layer = Dense(num_classes, activation='softmax')(Flatten()(attention_layer))

    model = Model(inputs=inputs, outputs=output_layer)
    model.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Train and evaluate the normal attention model
normal_attention_model = create_normal_attention_model()
normal_attention_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)
y_pred_normal = np.argmax(normal_attention_model.predict(X_test), axis=1)
accuracy_normal = accuracy_score(y_test, y_pred_normal)
print(f'Test Accuracy with normal attention: {accuracy_normal}')

In [ ]:
# Train and evaluate the clustered+autoencoder attention model
clustered_autoencoder_attention_model = create_clustered_autoencoder_attention_model()
clustered_autoencoder_attention_model.fit(encoded_embeddings, y_train, epochs=10, batch_size=32, validation_split=0.2)
# Ensure y_pred_clustered_autoencoder has the same number of samples as y_test
y_pred_clustered_autoencoder = np.argmax(clustered_autoencoder_attention_model.predict(encoded_embeddings), axis=1)
y_pred_clustered_autoencoder = y_pred_clustered_autoencoder[:y_test.shape[0]]

accuracy_clustered_autoencoder = accuracy_score(y_test, y_pred_clustered_autoencoder)
print(f'Test Accuracy with clustered+autoencoder attention: {accuracy_clustered_autoencoder}')

In [ ]:
import time
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from tensorflow.keras.callbacks import History




# Helper function to plot accuracy and loss and save figures
def plot_metrics_and_save(history, metric, figure_name):
    plt.plot(history.history[metric], label=f'Training {metric}')
    plt.plot(history.history[f'val_{metric}'], label=f'Validation {metric}')
    plt.xlabel('Epoch')
    plt.ylabel(metric)
    plt.legend()
    plt.savefig(figure_name)
    plt.show()

# Helper function to calculate space and time
def calculate_space_and_time(model, X_test, y_test):
    # Ensure that the number of samples in X_test matches the number of labels in y_test
    X_test = X_test[:y_test.shape[0]]

    start_time = time.time()
    y_pred = np.argmax(model.predict(X_test), axis=1)
    end_time = time.time()
    accuracy = accuracy_score(y_test, y_pred)
    execution_time = end_time - start_time
    space_used = model.count_params()
    
    return accuracy, execution_time, space_used


# Train and evaluate the normal attention model
normal_attention_model = create_normal_attention_model()
history_normal = normal_attention_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)
accuracy_normal, time_normal, space_normal = calculate_space_and_time(normal_attention_model, X_test, y_test)

# Train and evaluate the clustered+autoencoder attention model
clustered_autoencoder_attention_model = create_clustered_autoencoder_attention_model()
history_clustered_autoencoder = clustered_autoencoder_attention_model.fit(encoded_embeddings, y_train, epochs=10, batch_size=32, validation_split=0.2)
accuracy_clustered_autoencoder, time_clustered_autoencoder, space_clustered_autoencoder = calculate_space_and_time(clustered_autoencoder_attention_model, encoded_embeddings, y_test)

# Print results
print(f'Test Accuracy with normal attention: {accuracy_normal}')
print(f'Execution Time for normal attention: {time_normal} seconds')
print(f'Approximate Space Used by normal attention model: {space_normal} parameters\n')

print(f'Test Accuracy with clustered+autoencoder attention: {accuracy_clustered_autoencoder}')
print(f'Execution Time for clustered+autoencoder attention: {time_clustered_autoencoder} seconds')
print(f'Approximate Space Used by clustered+autoencoder attention model: {space_clustered_autoencoder} parameters\n')

# Plot training and validation accuracy and loss
plot_metrics_and_save(history_normal, 'accuracy', 'training_accuracy_normal.png')
plot_metrics_and_save(history_normal, 'loss', 'training_loss_normal.png')

plot_metrics_and_save(history_clustered_autoencoder, 'accuracy', 'training_accuracy_clustered_autoencoder.png')
plot_metrics_and_save(history_clustered_autoencoder, 'loss', 'training_loss_clustered_autoencoder.png')


In [ ]:
import os

# Helper function to plot accuracy and loss and save figures
def plot_metrics_and_save(history, metric, figure_name):
    plt.plot(history.history[metric], label=f'Training {metric}')
    plt.plot(history.history[f'val_{metric}'], label=f'Validation {metric}')
    plt.xlabel('Epoch')
    plt.ylabel(metric)
    plt.legend()
    plt.savefig(figure_name)
    plt.close()

# Helper function to save confusion matrix figure
def save_confusion_matrix(cm, labels, figure_name):
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=labels, yticklabels=labels)
    plt.title('Confusion Matrix')
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.savefig(figure_name)
    plt.close()

# Directory to save figures
figure_dir = 'figures'
os.makedirs(figure_dir, exist_ok=True)

# Train and evaluate models with normal attention and clustering attention
attention_layers = ['normal', 'clustering']
history_dict = {}

for attention_layer in attention_layers:
    print(f"Training model with {attention_layer} attention:")
    model = create_model(attention_layer)
    start_time = time.time()
    history = model.fit(X_train, y_train, epochs=5, batch_size=32, validation_split=0.1, verbose=0)
    end_time = time.time()

    # Save training and validation accuracy figures
    plot_metrics_and_save(history, 'accuracy', os.path.join(figure_dir, f'training_accuracy_{attention_layer}.png'))

    # Save training and validation loss figures
    plot_metrics_and_save(history, 'loss', os.path.join(figure_dir, f'training_loss_{attention_layer}.png'))

    # Save confusion matrix figure
    y_pred = np.argmax(model.predict(X_test), axis=1)
    cm = confusion_matrix(y_test, y_pred)
    save_confusion_matrix(cm, label_encoder.classes_, os.path.join(figure_dir, f'confusion_matrix_{attention_layer}.png'))

    # Time and Space
    execution_time = end_time - start_time
    print(f"Execution Time for {attention_layer} attention: {execution_time} seconds")
    print(f"Approximate Space Used by Model: {model.count_params()} parameters\n")

    # Save history for later analysis
    history_dict[attention_layer] = history


In [ ]:
clustered_embeddings.shape

In [ ]:
#with pca

import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Embedding, Flatten
from tensorflow.keras.optimizers import Adam

# Tokenize the text data
max_words = 6000
vectorizer = TfidfVectorizer(max_features=max_words, stop_words='english')
X_tfidf = vectorizer.fit_transform(df['clean'])
X_dense = X_tfidf.toarray()

# Encode categorical labels
label_encoder = LabelEncoder()
df['encoded_label'] = label_encoder.fit_transform(df['real'])
num_classes = len(label_encoder.classes_)

# Apply KMeans clustering on the tokenized text embeddings
embedding_dim = 32  # Adjust as needed
num_clusters = 2  # Number of clusters
kmeans = KMeans(n_clusters=num_clusters)
clustered_embeddings = kmeans.fit_predict(X_dense)

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_dense, df['encoded_label'], test_size=0.2, random_state=42)

# Apply PCA to the clustered embeddings
pca = PCA(n_components=embedding_dim)
pca.fit(X_dense)
pca_embeddings = pca.transform(X_dense)

# Define a simple transformer model with normal attention
def create_normal_attention_model():
    inputs = Input(shape=(X_train.shape[1],))
    embedding_layer = Embedding(input_dim=max_words, output_dim=embedding_dim)(inputs)
    q = Dense(embedding_dim)(embedding_layer)
    k = Dense(embedding_dim)(embedding_layer)
    v = Dense(embedding_dim)(embedding_layer)

    attention_layer = tf.keras.layers.Attention()([q, k, v])
    output_layer = Dense(num_classes, activation='softmax')(Flatten()(attention_layer))

    model = Model(inputs=inputs, outputs=output_layer)
    model.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Define a simple transformer model with clustered+PCA attention
def create_clustered_pca_attention_model():
    inputs = Input(shape=(embedding_dim,))
    q = Dense(embedding_dim)(inputs)
    k = Dense(embedding_dim)(inputs)
    v = Dense(embedding_dim)(inputs)

    # Clustering Attention Layer
    attention_layer = tf.keras.layers.Attention()([q, k, v])

    output_layer = Dense(num_classes, activation='softmax')(Flatten()(attention_layer))

    model = Model(inputs=inputs, outputs=output_layer)
    model.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Train and evaluate the normal attention model
normal_attention_model = create_normal_attention_model()
history_normal = normal_attention_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)
accuracy_normal = accuracy_score(y_test, np.argmax(normal_attention_model.predict(X_test), axis=1))

# Train and evaluate the clustered+PCA attention model
clustered_pca_attention_model = create_clustered_pca_attention_model()
history_clustered_pca = clustered_pca_attention_model.fit(pca_embeddings[:y_train.shape[0]], y_train, epochs=10, batch_size=32, validation_split=0.2)

accuracy_clustered_pca = accuracy_score(y_test, np.argmax(clustered_pca_attention_model.predict(pca.transform(X_test)), axis=1))

# Print results
print(f'Test Accuracy with normal attention: {accuracy_normal}')
print(f'Test Accuracy with clustered+PCA attention: {accuracy_clustered_pca}')




In [ ]:

# Plot training and validation accuracy
plt.plot(history_normal.history['accuracy'], label='Normal Training Accuracy')
plt.plot(history_normal.history['val_accuracy'], label='Normal Validation Accuracy')
plt.plot(history_clustered_pca.history['accuracy'], label='Clustered+PCA Training Accuracy')
plt.plot(history_clustered_pca.history['val_accuracy'], label='Clustered+PCA Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# Plot training and validation loss
plt.plot(history_normal.history['loss'], label='Normal Training Loss')
plt.plot(history_normal.history['val_loss'], label='Normal Validation Loss')
plt.plot(history_clustered_pca.history['loss'], label='Clustered+PCA Training Loss')
plt.plot(history_clustered_pca.history['val_loss'], label='Clustered+PCA Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
y_train.shape

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input, Embedding, Flatten
from tensorflow.keras.optimizers import Adam

# Define a custom Attention layer with clustering on k and q matrices
class ClusteringAttention(tf.keras.layers.Layer):
    def __init__(self, num_clusters, **kwargs):
        super(ClusteringAttention, self).__init__(**kwargs)
        self.num_clusters = num_clusters
        self.q_dense = Dense(embedding_dim)
        self.k_dense = Dense(embedding_dim)
        self.built = False

    def build(self, input_shape):
        if not self.built:
            super(ClusteringAttention, self).build(input_shape)
            self.built = True

    def call(self, inputs):
        q, k, v = inputs

        # Apply KMeans clustering on q and k matrices
        k_clustered = self.k_dense(k)
        q_clustered = self.q_dense(q)

        # Perform attention mechanism
        attention_scores = tf.matmul(q_clustered, k_clustered, transpose_b=True)
        attention_scores = tf.nn.softmax(attention_scores)

        # Apply attention scores to value matrix
        output = tf.matmul(attention_scores, v)
        return output

    def compute_output_shape(self, input_shape):
        return input_shape[-1]

# Define a simple transformer model with clustered attention
def create_model(attention_layer):
    inputs = Input(shape=(X.shape[1],))
    embedding_layer = Embedding(input_dim=max_words, output_dim=embedding_dim)(inputs)
    q = Dense(embedding_dim)(embedding_layer)
    k = Dense(embedding_dim)(embedding_layer)
    v = Dense(embedding_dim)(embedding_layer)

    if attention_layer == 'clustering':
        # Create clustering layers
        k_clustered_layer = Dense(num_clusters, use_bias=False)
        q_clustered_layer = Dense(num_clusters, use_bias=False)

        # Apply clustering layers to q and k
        k_clustered = k_clustered_layer(k)
        q_clustered = q_clustered_layer(q)

        attention_layer = ClusteringAttention(num_clusters)([q_clustered, k_clustered, v])
    else:
        attention_layer = tf.keras.layers.Attention()([q, k, v])

    output_layer = Dense(num_classes, activation='softmax')(Flatten()(attention_layer))

    model = Model(inputs=inputs, outputs=output_layer)
    model.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model


In [ ]:
# Define a custom Attention layer with clustering on k and q matrices
class ClusteringAttention(tf.keras.layers.Layer):
    def __init__(self, num_clusters, **kwargs):
        super(ClusteringAttention, self).__init__(**kwargs)
        self.num_clusters = num_clusters
        self.q_dense = Dense(embedding_dim)
        self.k_dense = Dense(embedding_dim)
        self.built = False

    def build(self, input_shape):
        if not self.built:
            super(ClusteringAttention, self).build(input_shape)
            self.built = True

    def call(self, inputs):
        q, k, v = inputs

        # Apply KMeans clustering on q and k matrices
        k_clustered = self.k_dense(k)
        q_clustered = self.q_dense(q)

        # Perform attention mechanism
        attention_scores = tf.matmul(q_clustered, k_clustered, transpose_b=True)
        attention_scores = tf.nn.softmax(attention_scores)

        # Apply attention scores to value matrix
        output = tf.matmul(attention_scores, v)
        return output

    def compute_output_shape(self, input_shape):
        return input_shape[-1]

# Define a simple transformer model with clustered attention
def create_model(attention_layer):
    inputs = Input(shape=(X.shape[1],))
    embedding_layer = Embedding(input_dim=max_words, output_dim=embedding_dim)(inputs)
    q = Dense(embedding_dim)(embedding_layer)
    k = Dense(embedding_dim)(embedding_layer)
    v = Dense(embedding_dim)(embedding_layer)

    if attention_layer == 'clustering':
        # Create clustering layers
        k_clustered_layer = Dense(num_clusters, use_bias=False)
        q_clustered_layer = Dense(num_clusters, use_bias=False)

        # Apply clustering layers to q and k
        k_clustered = k_clustered_layer(k)
        q_clustered = q_clustered_layer(q)

        attention_layer = ClusteringAttention(num_clusters)([q_clustered, k_clustered, v])
    else:
        attention_layer = tf.keras.layers.Attention()([q, k, v])

    output_layer = Dense(num_classes, activation='softmax')(Flatten()(attention_layer))

    model = Model(inputs=inputs, outputs=output_layer)
    model.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Train and evaluate models with normal attention and clustered attention
attention_layers = ['normal', 'clustering']
for attention_layer in attention_layers:
    model = create_model(attention_layer)
    model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)
    y_pred = np.argmax(model.predict(X_test), axis=1)
    accuracy = accuracy_score(y_test, y_pred)
    print(f'Test Accuracy with {attention_layer} attention: {accuracy}')

In [ ]:
encoded_embeddings.shape

In [ ]:
X_train.shape

In [ ]:
encoded_embeddings = encoded_embeddings.reshape(6000, 32)

In [ ]:
# Define a custom Attention layer with clustering on k and q matrices
class ClusteringAttention(tf.keras.layers.Layer):
    def __init__(self, num_clusters, **kwargs):
        super(ClusteringAttention, self).__init__(**kwargs)
        self.num_clusters = num_clusters
        self.q_dense = Dense(embedding_dim)
        self.k_dense = Dense(embedding_dim)
        self.built = False

    def build(self, input_shape):
        if not self.built:
            super(ClusteringAttention, self).build(input_shape)
            self.built = True

    def call(self, inputs):
        q, k, v = inputs

        # Apply KMeans clustering on q and k matrices
        k_clustered = self.k_dense(k)
        q_clustered = self.q_dense(q)

        # Perform attention mechanism
        attention_scores = tf.matmul(q_clustered, k_clustered, transpose_b=True)
        attention_scores = tf.nn.softmax(attention_scores)

        # Apply attention scores to value matrix
        output = tf.matmul(attention_scores, v)
        return output

    def compute_output_shape(self, input_shape):
        return input_shape[-1]

# Define a simple transformer model with clustered attention

# Preprocess the text data
max_words = 10000  # Maximum number of words to tokenize
max_sequence_length = 50  # Maximum sequence length for padding
embedding_dim = 100  # Dimension of word embeddings
num_clusters = 100  # Number of clusters for k-means


inputs = Input(shape=(X.shape[1],))
embedding_layer = Embedding(input_dim=max_words, output_dim=embedding_dim)(inputs)
q = Dense(embedding_dim)(embedding_layer)
k = Dense(embedding_dim)(embedding_layer)
v = Dense(embedding_dim)(embedding_layer)

# Create clustering layers
k_clustered_layer = Dense(num_clusters, use_bias=False)
q_clustered_layer = Dense(num_clusters, use_bias=False)

# Apply clustering layers to q and k
k_clustered = k_clustered_layer(k)
q_clustered = q_clustered_layer(q)

attention_layer = ClusteringAttention(num_clusters)([q_clustered, k_clustered, v])
output_layer = Dense(num_classes, activation='softmax')(Flatten()(attention_layer))

model = Model(inputs=inputs, outputs=output_layer)
model.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [ ]:
print(model.summary())

In [ ]:
from keras.utils.vis_utils import plot_model
plot_model(clustered_autoencoder_attention_model, to_file='aumodel_plot.png', show_shapes=True, show_layer_names=True)

In [ ]:
# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=16, validation_split=0.2)

In [ ]:
# Evaluate on test data
y_pred = np.argmax(model.predict(X_test), axis=1)
accuracy = accuracy_score(y_test, y_pred)
print(f'Test Accuracy: {accuracy}')

In [ ]:
visualkeras.layered_view(model)

In [ ]:
#combination of Clustered and normal attention
# Tokenize the text data

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Embedding, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

max_words = 10000
tokenizer = Tokenizer(num_words=max_words, oov_token='<OOV>')
tokenizer.fit_on_texts(df['clean'])
X = tokenizer.texts_to_sequences(df['clean'])
X = pad_sequences(X)

# Encode categorical labels
label_encoder = LabelEncoder()
df['encoded_label'] = label_encoder.fit_transform(df['real'])
num_classes = len(label_encoder.classes_)

# Apply KMeans clustering on the tokenized text embeddings
embedding_dim = 32  # Adjust as needed
num_clusters = 2  # Number of clusters
kmeans = KMeans(n_clusters=num_clusters)
embeddings = tf.keras.layers.Embedding(input_dim=max_words, output_dim=embedding_dim)(X)
flat_embeddings = tf.keras.layers.Flatten()(embeddings)
clustered_embeddings = kmeans.fit_predict(flat_embeddings)

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, df['encoded_label'], test_size=0.2, random_state=42)


In [ ]:
# Define a custom Attention layer with clustering on k and q matrices
class ClusteringAttention(tf.keras.layers.Layer):
    def __init__(self, num_clusters, **kwargs):
        super(ClusteringAttention, self).__init__(**kwargs)
        self.num_clusters = num_clusters
        self.q_dense = Dense(embedding_dim)
        self.k_dense = Dense(embedding_dim)
        self.built = False

    def build(self, input_shape):
        if not self.built:
            super(ClusteringAttention, self).build(input_shape)
            self.built = True

    def call(self, inputs):
        q, k, v = inputs

        # Apply KMeans clustering on q and k matrices
        k_clustered = self.k_dense(k)
        q_clustered = self.q_dense(q)

        # Perform attention mechanism
        attention_scores = tf.matmul(q_clustered, k_clustered, transpose_b=True)
        attention_scores = tf.nn.softmax(attention_scores)

        # Apply attention scores to value matrix
        output = tf.matmul(attention_scores, v)
        return output

    def compute_output_shape(self, input_shape):
        return input_shape[-1]

# Define a simple transformer model with clustered attention
def create_model(attention_layer):
    inputs = Input(shape=(X.shape[1],))
    embedding_layer = Embedding(input_dim=max_words, output_dim=embedding_dim)(inputs)
    q = Dense(embedding_dim)(embedding_layer)
    k = Dense(embedding_dim)(embedding_layer)
    v = Dense(embedding_dim)(embedding_layer)

    if attention_layer == 'clustering':
        # Create clustering layers
        k_clustered_layer = Dense(num_clusters, use_bias=False)
        q_clustered_layer = Dense(num_clusters, use_bias=False)

        # Apply clustering layers to q and k
        k_clustered = k_clustered_layer(k)
        q_clustered = q_clustered_layer(q)

        attention_layer = ClusteringAttention(num_clusters)([q_clustered, k_clustered, v])
    else:
        attention_layer = tf.keras.layers.Attention()([q, k, v])

    output_layer = Dense(num_classes, activation='softmax')(Flatten()(attention_layer))

    model = Model(inputs=inputs, outputs=output_layer)
    model.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
# Train and evaluate models with normal attention and clustered attention
attention_layers = ['normal', 'clustering']
for attention_layer in attention_layers:
    model = create_model(attention_layer)
    model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)
    y_pred = np.argmax(model.predict(X_test), axis=1)
    accuracy = accuracy_score(y_test, y_pred)
    print(f'Test Accuracy with {attention_layer} attention: {accuracy}')

In [ ]:
import time
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
from tensorflow.keras.callbacks import History

# Helper function to plot accuracy and loss and save figures
def plot_metrics_and_save(history, metric, figure_name):
    plt.plot(history.history[metric], label=f'Training {metric}')
    plt.plot(history.history[f'val_{metric}'], label=f'Validation {metric}')
    plt.xlabel('Epoch')
    plt.ylabel(metric)
    plt.legend()
    plt.savefig(figure_name)
    plt.close()

# Helper function to save confusion matrix figure
def save_confusion_matrix(cm, labels, figure_name):
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=labels, yticklabels=labels)
    plt.title('Confusion Matrix')
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.savefig(figure_name)
    plt.close()

# Train and evaluate models with normal attention and clustering attention
attention_layers = ['normal', 'clustering']
history_dict = {}

for attention_layer in attention_layers:
    print(f"Training model with {attention_layer} attention:")
    model = create_model(attention_layer)
    start_time = time.time()
    history = model.fit(X_train, y_train, epochs=5, batch_size=32, validation_split=0.1, verbose=0)
    end_time = time.time()
    
    y_pred = np.argmax(model.predict(X_test), axis=1)
    accuracy = accuracy_score(y_test, y_pred)
    print(f'Test Accuracy with {attention_layer} attention: {accuracy}')
    
    # Save training and validation accuracy figures
    plot_metrics_and_save(history, 'accuracy', f'training_accuracy_{attention_layer}.png')
    
    # Save training and validation loss figures
    plot_metrics_and_save(history, 'loss', f'training_loss_{attention_layer}.png')
    
    # Save confusion matrix figure
    cm = confusion_matrix(y_test, y_pred)
    save_confusion_matrix(cm, label_encoder.classes_, f'confusion_matrix_{attention_layer}.png')

    # Time and Space
    execution_time = end_time - start_time
    print(f"Execution Time for {attention_layer} attention: {execution_time} seconds")
    print(f"Approximate Space Used by Model: {model.count_params()} parameters\n")
    
    # Save history for later analysis
    history_dict[attention_layer] = history


In [ ]:
from tensorflow.keras.utils import plot_model

# Function to create and plot the model
def create_and_plot_model(attention_layer, figure_name):
    model = create_model(attention_layer)
    plot_model(model, to_file=figure_name, show_shapes=True, show_layer_names=True)

# Plot normal attention model
create_and_plot_model('normal', 'model_plot_normal.png')

# Plot clustering attention model
create_and_plot_model('clustering', 'model_plot_clustering.png')


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Embedding, LSTM, Dense
from sklearn.cluster import KMeans


# Preprocess the text data
max_words = 10000  # Maximum number of words to tokenize
max_sequence_length = 100  # Maximum sequence length for padding
embedding_dim = 100  # Dimension of word embeddings
num_clusters = 100  # Number of clusters for k-means

tokenizer = Tokenizer(num_words=max_words, oov_token='<OOV>')
tokenizer.fit_on_texts(df['clean'])
X = tokenizer.texts_to_sequences(df['clean'])
X = pad_sequences(X, maxlen=max_sequence_length)

# Convert labels to numerical values (assuming 'pos' is 1 and 'neg' is 0)
labels = df['Sentiment'].apply(lambda x: 1 if x == 'positive' else 0)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.2, random_state=42)


In [ ]:
# Build and train your LSTM model
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=max_sequence_length))
model.add(LSTM(units=128))
model.add(Dense(units=1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=5, batch_size=32, validation_split=0.1)

In [ ]:
# Evaluate the new model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test accuracy: {accuracy * 100:.2f}%')

In [ ]:
# Extract embeddings from the penultimate layer
penultimate_layer_model = Model(inputs=model.input, outputs=model.layers[-2].output)
embeddings = penultimate_layer_model.predict(X)

In [ ]:
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from tensorflow.keras.models import Model
#import plotly.express as px

In [ ]:
# Apply t-SNE to reduce the dimensions of the embeddings to 2D
tsne = TSNE(n_components=2, random_state=42)
embeddings_2d = tsne.fit_transform(embeddings)

# Create a scatter plot for visualization
plt.figure(figsize=(10, 8))
plt.scatter(embeddings_2d[:, 0], embeddings_2d[:, 1], c=labels, cmap='coolwarm', s=10)  # 'labels' represent the true labels of your data points
plt.colorbar()  # Add a colorbar to show the mapping of colors to labels
plt.xlabel('t-SNE Dimension 1')
plt.ylabel('t-SNE Dimension 2')
plt.title('t-SNE Visualization of Penultimate Layer Embeddings')
plt.show()